Demo

In [ ]:
# Auto Encoding (for NLU purpose)

In [ ]:
# Hugging Face

In [ ]:
 # installing required libraries
!pip install transformers
!pip install sentencepiece
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.8 MB/s 
     |████████████████████████████████| 7.6 MB 61.4 MB/s 
     |████████████████████████████████| 163 kB 66.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 11.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 40.8 MB/s 
     |████████████████████████████████| 287 kB 51.2 MB/s 


In [ ]:
# importing required libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from tqdm import tqdm, trange
from ast import literal_eval
import json
import nltk
from nltk.tokenize import LineTokenizer, sent_tokenize
nltk.download('punkt')
import re
import contractions
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# mounting google drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
# This function processes the text so that it can be used to produce meaningful embeddings
def processText(text):
    
    # Using the line tokenizer to split using new line character (\n) and discarding the blank lines
    if text is not None:
        
        lines = LineTokenizer(blanklines = 'discard').tokenize(text)

        sentences = []

        # For each line found above we are doing sentence tokenization
        for line in lines:
            currentSentences = sent_tokenize(line)
            # sentences.extend([sentence for sentence in currentSentences ])
            for sentence in currentSentences:
                currentText = str(sentence.encode("ascii", "ignore").decode().strip())
                sentences.extend([currentText])
                text = '. '.join([processSentence(sentence) for sentence in sentences])
    return text


# This function processes the sentence so that it can be used to produce meaningful embeddings
def processSentence(text):
    # Removing HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Removing new line characters
    text = text.replace('\\n', '')
    # Removing urls
    # text = re.sub('[^ ]+\.[^ ]+', ' ', text)
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''," ",text)
    # Removing email addresses
    text = re.sub(r'''\S*@\S*\s?''', " ", text)
    # Expanding the contractions
    text = contractions.fix(text)
    return text

In [ ]:
df = pd.read_csv("/content/drive/My Drive/training.1600000.processed.noemoticon.csv", encoding='mac_roman', header = None)
df = df.sample(1600)
df[0] = df[0].apply(lambda x: 0 if x == 0 else 1)
df

,0,1,2,3,4,5
338340,0,2014404655,Wed Jun 03 01:25:11 PDT 2009,NO_QUERY,Spidersamm,@sonnyjoeflangan awww are you shitted cos you ...
714913,0,2259274003,Sat Jun 20 17:18:12 PDT 2009,NO_QUERY,hotcheat,whew!!! just finished washing tons of dishes!!...
1456178,1,2063474536,Sun Jun 07 02:48:24 PDT 2009,NO_QUERY,richontwitter,@loopymups @c2xu @ChrisBeach I know who my dr...
1390142,1,2053191867,Sat Jun 06 03:38:03 PDT 2009,NO_QUERY,invinciblewest,@piamagalona hey ms. P! ur such a FAB mommie!!
723359,0,2261765955,Sat Jun 20 21:18:47 PDT 2009,NO_QUERY,MisterOscar,@xiDamagex Yeah I was gonna see it last week b...
...,...,...,...,...,...,...
325325,0,2007080447,Tue Jun 02 12:07:01 PDT 2009,NO_QUERY,vikkeh,@heypunka enders is shit now!
282043,0,1992500626,Mon Jun 01 08:51:17 PDT 2009,NO_QUERY,domwilson,took my boo to the repair shop
954969,1,1824937777,Sun May 17 03:36:33 PDT 2009,NO_QUERY,alangurbutt,@inspiritu &quot;[...] Nice park. Gorgeous day...
145347,0,1882191438,Fri May 22 06:24:41 PDT 2009,NO_QUERY,jona_thin,Why is TweetDeck not working anymore


In [ ]:
comments = list(df[5])
labels = list(df[0])

In [ ]:
# tokenizing using RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encodings = tokenizer.batch_encode_plus(comments, max_length = 512, pad_to_max_length = True) 

print('tokenizer outputs: ', encodings.keys())

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [ ]:
# getting tokenized and encoded sentences and attention masks
input_ids = encodings['input_ids'] 
attention_masks = encodings['attention_mask'] 

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks, random_state = 303, test_size = 500)

print("x_train", len(train_inputs))
print("x_vad", len(validation_inputs))
print("y_train", len(train_labels))
print("y_vad", len(validation_labels))

x_train 1100
x_vad 500
y_train 1100
y_vad 500


In [ ]:
# Converting all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# Creating an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = 16)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler = validation_sampler, batch_size = 16)

In [ ]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)
model.cuda()

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
# setting custom optimization parameters.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [ {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01}, {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}]

optimizer = AdamW(optimizer_grouped_parameters, lr = 2e-5, correct_bias = True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# Train Model
torch.cuda.memory_summary(device = None, abbreviated = False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  488175 KB |  488175 KB |  488175 KB |       0 B  |\n|       from large pool |  487680 KB |  487680 KB |  487680 KB |       0 B  |\n|       from small pool |     495 KB |     495 KB |     495 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  488175 KB |  488175 KB |  488175 KB |       0 B  |\n|       from large pool |  487680 KB |  487680 KB |

In [ ]:
# Store our loss and accuracy for plotting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc = "Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader): 
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    print(b_input_ids)
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    print("output type: ", type(outputs))
    loss = outputs[0].float()
    #print("loss type: ", type(loss.item()))
    logits = outputs[1].float()

    # Forward pass for multilabel classification
    #outputs = model(b_input_ids, attention_mask = b_input_mask)
    #logits = outputs[0]
    #loss_func = BCEWithLogitsLoss() 
    #loss = loss_func(logits.view(-1,2), b_labels.type_as(logits).view(-1,2)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    #train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()

    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

  ###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds, true_labels, pred_labels, tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    print(b_input_ids)
    with torch.no_grad():
      
      # Forward pass
      outs = model(b_input_ids, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  print(true_bools)
  print(pred_bools)
  #val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  #val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  #print('F1 Validation Accuracy: ', val_f1_accuracy)
  #print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

tensor([[    0,  1039, 23471,  ...,     1,     1,     1],
        [    0,  1039,  4992,  ...,     1,     1,     1],
        [    0, 21719,    57,  ...,     1,     1,     1],
        ...,
        [    0,   354,    24,  ...,     1,     1,     1],
        [    0, 12690, 15894,  ...,     1,     1,     1],
        [    0,  1039,  1001,  ...,     1,     1,     1]], device='cuda:0')
output type:  <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
tensor([[    0,   100,   437,  ...,     1,     1,     1],
        [    0, 12891,    15,  ...,     1,     1,     1],
        [    0,   354,   816,  ...,     1,     1,     1],
        ...,
        [    0,  1039, 22448,  ...,     1,     1,     1],
        [    0,  1039, 42084,  ...,     1,     1,     1],
        [    0, 37188,    14,  ...,     1,     1,     1]], device='cuda:0')
output type:  <class 'transformers.modeling_outputs.SequenceClassifierOutput'>
tensor([[    0,  2527,  7727,  ...,     1,     1,     1],
        [    0,  1039, 305

Epoch:  33%|███▎      | 1/3 [02:30<05:00, 150.05s/it]

tensor([[    0,  5320,  3239,  ...,     1,     1,     1],
        [    0,  1039,   534,  ...,     1,     1,     1],
        [    0,  6785, 13356,  ...,     1,     1,     1],
        [    0, 37188, 15192,  ...,     1,     1,     1]], device='cuda:0')
[False, True, True, True, False, True, False, False, True, True, True, False, True, False, True, True, False, True, False, False, True, False, True, False, True, False, True, False, True, False, True, False, False, False, False, True, True, True, True, True, True, False, True, True, False, False, True, True, False, True, False, False, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, True, False, True, True, True, True, False, True, True, False, False, False, True, True, False, True, True, True, False, True, False, False, True, False, False, False, False, False, False, False, False, True, True, False, True, False, False, True, False, True, False, False, False, True, False, True, False, False

Epoch:  67%|██████▋   | 2/3 [05:07<02:34, 154.64s/it]

tensor([[    0,  5320,  3239,  ...,     1,     1,     1],
        [    0,  1039,   534,  ...,     1,     1,     1],
        [    0,  6785, 13356,  ...,     1,     1,     1],
        [    0, 37188, 15192,  ...,     1,     1,     1]], device='cuda:0')
[False, True, True, True, False, True, False, False, True, True, True, False, True, False, True, True, False, True, False, False, True, False, True, False, True, False, True, False, True, False, True, False, False, False, False, True, True, True, True, True, True, False, True, True, False, False, True, True, False, True, False, False, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, True, False, True, True, True, True, False, True, True, False, False, False, True, True, False, True, True, True, False, True, False, False, True, False, False, False, False, False, False, False, False, True, True, False, True, False, False, True, False, True, False, False, False, True, False, True, False, False

Epoch: 100%|██████████| 3/3 [07:48<00:00, 156.25s/it]

tensor([[    0,  5320,  3239,  ...,     1,     1,     1],
        [    0,  1039,   534,  ...,     1,     1,     1],
        [    0,  6785, 13356,  ...,     1,     1,     1],
        [    0, 37188, 15192,  ...,     1,     1,     1]], device='cuda:0')
[False, True, True, True, False, True, False, False, True, True, True, False, True, False, True, True, False, True, False, False, True, False, True, False, True, False, True, False, True, False, True, False, False, False, False, True, True, True, True, True, True, False, True, True, False, False, True, True, False, True, False, False, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, True, False, True, True, True, True, False, True, True, False, False, False, True, True, False, True, True, True, False, True, False, False, True, False, False, False, False, False, False, False, False, True, True, False, True, False, False, True, False, True, False, False, False, True, False, True, False, False

In [ ]:
sample_text = ["@piamagalona hey ms. P! ur such a FAB mommie!!"]
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encodings = tokenizer(sample_text, pad_to_max_length = True) 
input_ids = torch.tensor(encodings['input_ids'])
attention_masks = torch.tensor(encodings['attention_mask'] )
input_ids

tensor([[    0,  1039,   642,  6009,  1073,   337,  4488, 17232, 43601,     4,
           221,   328, 11540,   215,    10,   274,  4546,  3795, 23738, 12846,
             2]])

In [ ]:
outs = model(input_ids.cuda(), attention_mask = attention_masks.cuda())
b_logit_pred = outs[0]
pred_label = torch.sigmoid(b_logit_pred)
print(pred_label)

tensor([[0.1422, 0.9440]], device='cuda:0', grad_fn=<SigmoidBackward0>)


Homework

In [ ]:
# Perform Multiclass Classification Using Roberta (You should have minimum of 3 classes)